#### Finding popular / unpopular keywords

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math

In [2]:
df_in = pd.read_excel( '../data/input/OnlineNewsPopularity.xlsx')[["Id", " shares"]]
df_in["shares"] = df_in[" shares"]
df_in = df_in.drop( columns=[" shares"], axis=1)

df_in["shares_no_clip"] = df_in["shares"]

# for df_in alone clip it to 95th percentile
print( (df_in.shares>9500).astype(int).sum())
df_in.shares[ df_in.shares > 9500] = 9500
print( (df_in.shares>9500).astype(int).sum())

print( df_in.shape)
print( (df_in.shares > 9500).astype(int).sum() )
print( (df_in.shares_no_clip > 9500).astype(int).sum() )

384
0
(7795, 3)
0
384


In [3]:
df_keywords = pd.read_excel( '../data/output/2_keywords_list.xlsx')
df_keywords.shape

(7795, 3)

#### For each keyword, get the following
* Number of articles it appears
* Average shares
* Min number of shares it received
* Max number of shares it received
* Average shares (no clipping of 95%)
* Min number of shares it received (no clipping of 95%)
* Max number of shares it received (no clipping of 95%)

In [4]:
df_popular = pd.DataFrame( columns=["keyword", "article_count", \
                                    "shares", "min_shares", "max_shares", \
                                    "shares_no_clip", "min_shares_no_clip", "max_shares_no_clip" \
                                    ])
df_popular.set_index( "keyword", inplace=True)
df_popular = df_popular.astype({'article_count':int, \
                                'shares':int, 'min_shares':int, 'max_shares':int, \
                                'shares_no_clip':int, 'min_shares_no_clip':int, 'max_shares_no_clip':int \
                               }) 
df_popular

,article_count,shares,min_shares,max_shares,shares_no_clip,min_shares_no_clip,max_shares_no_clip
keyword,,,,,,,


In [5]:
for index, row in df_keywords.iterrows():
    id = row.Id
    keywords = row.keywords
    shares = df_in.at[index, "shares"]
    shares_no_clip = df_in.at[index, "shares_no_clip"]
    
    for keyword in keywords.split(", "):
        if df_popular.index.contains(keyword):
            df_popular.at[keyword, "article_count"] += 1
            df_popular.at[keyword, "shares"] += shares
            df_popular.at[keyword, "shares_no_clip"] += shares_no_clip
            
            if shares > df_popular.at[keyword, "max_shares"]:
                df_popular.at[keyword, "max_shares"] = shares
                
            if shares < df_popular.at[keyword, "min_shares"]:
                df_popular.at[keyword, "min_shares"] = shares
                
            if shares_no_clip > df_popular.at[keyword, "max_shares_no_clip"]:
                df_popular.at[keyword, "max_shares_no_clip"] = shares_no_clip
                
            if shares_no_clip < df_popular.at[keyword, "min_shares_no_clip"]:
                df_popular.at[keyword, "min_shares_no_clip"] = shares_no_clip
        else:
            df_popular.at[keyword, "article_count"] = 1
            df_popular.at[keyword, "shares"] = shares
            df_popular.at[keyword, "min_shares"] = shares
            df_popular.at[keyword, "max_shares"] = shares
            
            df_popular.at[keyword, "shares_no_clip"] = shares_no_clip
            df_popular.at[keyword, "min_shares_no_clip"] = shares_no_clip
            df_popular.at[keyword, "max_shares_no_clip"] = shares_no_clip

df_popular.shape

(6106, 7)

### Find the maximum values

In [6]:
df_popular["avg_shares"] = ( df_popular["shares"] / df_popular["article_count"]).astype(int)
df_popular["avg_shares_no_clip"] = ( df_popular["shares_no_clip"] / df_popular["article_count"]).astype(int)
df_popular.head(5)

,article_count,shares,min_shares,max_shares,shares_no_clip,min_shares_no_clip,max_shares_no_clip,avg_shares,avg_shares_no_clip
keyword,,,,,,,,,
north-korea,46.0,99910.0,582.0,9500.0,101010.0,582.0,10600.0,2171,2195
uncategorized,7787.0,17320868.0,5.0,9500.0,23585567.0,5.0,284700.0,2224,3028
us-world,2374.0,5176343.0,22.0,9500.0,7083642.0,22.0,284700.0,2180,2983
world,1516.0,3373012.0,28.0,9500.0,5086712.0,28.0,284700.0,2224,3355
detainees,2.0,1758.0,658.0,1100.0,1758.0,658.0,1100.0,879,879


In [7]:
((df_popular["avg_shares"] - df_popular["avg_shares_no_clip"]) != 0).astype(int).sum()

841

### Keywords to ignore

* Max used keywords - anything > 2000 articles
    * uncategorized appears in almost all the entries
    * us-world appears in almost two thirds of the entries
* Anything appeared in lesser than or equal of 3 articles 

###### We will consider only the keywords which appeared in atleast 4 articles and atmost 2000 articles

In [8]:
#df_popular = df_popular[ (df_popular.article_count < 2000) & (df_popular.article_count > 3) ]
#df_popular.shape

In [9]:
df_popular.drop( columns=["shares", "shares_no_clip"], axis=1, inplace=True)
df_popular.shape

(6106, 7)

In [10]:
df_popular.to_excel('../data/output/2_keywords_popularity.xlsx', index=True)

### Predicting shares based on keywords popularity

In [1]:
import pandas as pd
import math

from KeywordsPopularity import KeyPop
keyPop = KeyPop()

In [2]:
# Training RMSE
df_in = pd.read_excel( '../data/input/OnlineNewsPopularity.xlsx')[["Id", " shares"]]
df_in["shares"] = df_in[" shares"]
df_in = df_in.drop( columns=[" shares"], axis=1)
df_in["shares_no_clip"] = df_in["shares"]
df_in.shares[ df_in.shares > 9500] = 9500

df_keywords = pd.read_excel( '../data/output/2_keywords_list.xlsx')[["Id", "keywords"]]

In [3]:
df_merged = df_in.merge( df_keywords)
df_merged.head()

,Id,shares,shares_no_clip,keywords
0,1,1100,1100,"north-korea, uncategorized, us-world, world, d..."
1,2,1100,1100,"apple, visa, mobile-payments, american-express..."
2,3,1000,1000,"australia, uncategorized, us-world, sports, ma..."
3,4,822,822,"australia, bmw, uncategorized, us-world, tony-..."
4,5,841,841,"q-a, australia, uncategorized, tv, us-world, j..."


In [4]:
total_squared_error = 0.0
total_squared_error_no_clip = 0.0

df_out = pd.DataFrame(columns=["Id", "kw_avg_avg", "kw_min_avg", "kw_max_avg", \
                                   "kw_avg_avg_no_clip", "kw_min_avg_no_clip", "kw_max_avg_no_clip"])

df_out.astype({"Id":int, "kw_avg_avg":int, "kw_min_avg":int, "kw_max_avg":int, \
                "kw_avg_avg_no_clip":int, "kw_min_avg_no_clip":int, "kw_max_avg_no_clip":int})

for index, row in df_merged.iterrows():
    shares = row.shares
    shares_no_clip = row.shares_no_clip
    keywords = row.keywords
    
    predicted_shares = keyPop.predict_shares(keywords)
    
    df_out.at[index, "Id"] = row.Id
    df_out.at[index, "kw_avg_avg"] = predicted_shares.avg_avg
    df_out.at[index, "kw_min_avg"] = predicted_shares.min_avg
    df_out.at[index, "kw_max_avg"] = predicted_shares.max_avg
    df_out.at[index, "kw_avg_avg_no_clip"] = predicted_shares.avg_avg_no_clip
    df_out.at[index, "kw_min_avg_no_clip"] = predicted_shares.min_avg_no_clip
    df_out.at[index, "kw_max_avg_no_clip"] = predicted_shares.max_avg_no_clip
    
    total_squared_error += abs( predicted_shares.avg_avg - shares ) ** 2
    total_squared_error_no_clip += abs( predicted_shares.avg_avg_no_clip - shares_no_clip ) ** 2
    
mean_squared_error = total_squared_error / df_merged.shape[0]
rmse = math.sqrt( mean_squared_error)

mean_squared_error_no_clip = total_squared_error_no_clip / df_merged.shape[0]
rmse_no_clip = math.sqrt( mean_squared_error_no_clip)

df_out.to_excel('../data/output/2_keywords_popularity_prediction.xlsx', index=False)

print( rmse)
print( rmse_no_clip)

2048.4142168009002
7433.599397132549


### [Kannan]: calculating additional columns related to keywords

In [6]:
df_stats=pd.read_excel('../data/output/2_keywords_popularity.xlsx')
df_stats.drop(['article_count'],axis=1,inplace=True)
df_stats.head()

,keyword,min_shares,max_shares,avg_shares
0,north-korea,582,9500,2171
1,world,28,9500,2224
2,apple,43,9500,2219
3,mobile-payments,613,9500,2119
4,mastercard,859,9500,2971


In [15]:
df_stats.shape

(1203, 4)

In [7]:
df_kw=pd.read_excel('../data/output/2_keywords_list.xlsx')
df_kw.head()

,Id,keywords
0,1,"north-korea, uncategorized, us-world, world, d..."
1,2,"apple, visa, mobile-payments, american-express..."
2,3,"australia, uncategorized, us-world, sports, ma..."
3,4,"australia, bmw, uncategorized, us-world, tony-..."
4,5,"q-a, australia, uncategorized, tv, us-world, j..."


In [5]:
df_kw.loc[0,'keywords']

'north-korea, uncategorized, us-world, world, detainees, matthew-miller, jeffrey-fowle'

In [14]:
kws=df_kw.loc[0,'keywords']
list_kw=kws.split(',')
for kw in list_kw:
    kw=kw.strip()
    stats=df_stats[df_stats.keyword==kw]
    print(kw,stats)

north-korea        keyword  min_shares  max_shares  avg_shares
0  north-korea         582        9500        2171
uncategorized Empty DataFrame
Columns: [keyword, min_shares, max_shares, avg_shares]
Index: []
us-world Empty DataFrame
Columns: [keyword, min_shares, max_shares, avg_shares]
Index: []
world   keyword  min_shares  max_shares  avg_shares
1   world          28        9500        2224
detainees Empty DataFrame
Columns: [keyword, min_shares, max_shares, avg_shares]
Index: []
matthew-miller Empty DataFrame
Columns: [keyword, min_shares, max_shares, avg_shares]
Index: []
jeffrey-fowle Empty DataFrame
Columns: [keyword, min_shares, max_shares, avg_shares]
Index: []
